# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

df.head


<bound method NDFrame.head of        loan_amnt  int_rate  installment home_ownership  annual_inc  \
0        10500.0    0.1719       375.35           RENT     66000.0   
1        25000.0    0.2000       929.09       MORTGAGE    105000.0   
2        20000.0    0.2000       529.88       MORTGAGE     56000.0   
3        10000.0    0.1640       353.55           RENT     92000.0   
4        22000.0    0.1474       520.39       MORTGAGE     52000.0   
...          ...       ...          ...            ...         ...   
68812    10000.0    0.1502       346.76           RENT     26000.0   
68813    12000.0    0.2727       368.37           RENT     63000.0   
68814     5000.0    0.1992       185.62       MORTGAGE     52000.0   
68815    40000.0    0.0646      1225.24       MORTGAGE    520000.0   
68816    16000.0    0.1131       350.36       MORTGAGE     72000.0   

      verification_status   issue_d loan_status pymnt_plan    dti  ...  \
0         Source Verified  Mar-2019    low_risk        

## Split the Data into Training and Testing

In [5]:
# Create our features
X_df = df.drop(columns=['loan_status'])
X = pd.get_dummies(X_df, columns=["home_ownership","verification_status","issue_d",
                                  "pymnt_plan","initial_list_status","next_pymnt_d","debt_settlement_flag",
                                  "application_type","hardship_flag"], drop_first = True)

# Create our target
y = df['loan_status'].to_frame()

In [6]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.526309,0.106747,0.357659,0.373992,0.148001,0.451066,0.177238,0.876121,0.616839,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.499311,0.308793,0.479314,0.483865,0.355104,0.497603,0.381873,0.329446,0.486161,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
# Check the balance of our target values
# YOUR CODE HERE
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
# YOUR CODE HERE
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()



In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)


In [11]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier

brfcm = BalancedRandomForestClassifier(n_estimators =1000, random_state=1)
model = brfcm.fit(X_train_scaled, y_train)
BalancedRandomForestClassifier()

BalancedRandomForestClassifier()

In [13]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
predictions = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions)


0.8162470639899118

In [14]:
# Display the confusion matrix
# YOUR CODE HERE
cm = confusion_matrix(y_test, predictions)
cm

array([[   62,    25],
       [ 1372, 15746]], dtype=int64)

In [15]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.71      0.92      0.08      0.81      0.64        87
   low_risk       1.00      0.92      0.71      0.96      0.81      0.67     17118

avg / total       0.99      0.92      0.71      0.95      0.81      0.67     17205



In [16]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = model.feature_importances_

sorted(zip(model.feature_importances_, X.columns), reverse=True)[:10]

[(0.07440136919465735, 'total_rec_prncp'),
 (0.061437291161306265, 'total_rec_int'),
 (0.06130032888752441, 'total_pymnt_inv'),
 (0.06055470587529498, 'last_pymnt_amnt'),
 (0.05853446188877031, 'total_pymnt'),
 (0.028841819188718195, 'int_rate'),
 (0.019184048857469934, 'issue_d_Jan-2019'),
 (0.018344422390578374, 'dti'),
 (0.017900068474105562, 'installment'),
 (0.01592192753388305, 'il_util')]

### Easy Ensemble Classifier

In [17]:
# Train the Classifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(base_estimator=None, n_estimators=100, random_state=1, 
                                   replacement=False)

eec = eec.fit(X_train_scaled, y_train)

In [18]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
predictions = eec.predict(X_test_scaled)
accuracy = balanced_accuracy_score(y_test, predictions)
print(f'Balanced Accuracy Score: {accuracy}')

Balanced Accuracy Score: 0.9263912558266958


In [19]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, predictions)

array([[   79,     8],
       [  946, 16172]], dtype=int64)

In [20]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.91      0.94      0.14      0.93      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.

In [21]:
#1. The easy ensemble classifier had the best balanced accuarcy score at 0.92

#2. The easy ensemble classifier also had the best recall score of 0.94

#3. The easy ensemble classifier had the highest Geometric score of 0.93

